# How to Use: `calc_humbl_channel`


## Log into OpenBB

In [1]:
%load_ext autoreload
%autoreload 2

from openbb import obb

from humbldata.core.utils.env import Env
from humbldata.core.utils.openbb_helpers import obb_login

# obb_login()
obb.account.login(pat=Env().OBB_PAT, remember_me=True)


## Collect Data

In [3]:
from humbldata.toolbox.technical.humbl_channel.helpers import add_window_index
from humbldata.toolbox.toolbox_helpers import log_returns

symbols = ["AAPL", "PCT", "NVDA", "SMCI", "TSLA", "AMD", "SPY", "GLD", "BTC-USD", "ETH-USD"]
rr_list = ["MSFT", "AAPL", "AMZN", "META", "GOOGL", "NFLX", "TSLA", "NVDA"]
sector_symbols = [
    "XLF",
    "XLU",
    "XLK",
    "XLE",
    "XLI",
    "XLV",
    "XLY",
    "XLP",
    "XLB",
    "XLRE",
    "XTL"
]
simple= ["AAPL", "AMZN", "AMD"]


data = (
    obb.equity.price.historical(
        simple,
        provider="yfinance",
        start_date="2000-01-01",
        end_date="2024-01-01",
        adjustment="splits_and_dividends",
    ).to_polars()
).drop(["dividends"])

data = log_returns(data, _column_name="close")


ColumnNotFoundError: "dividends" not found

Resolved plan until failure:

	---> FAILED HERE RESOLVING THIS_NODE <---
DF ["date", "open", "high", "low", ...]; PROJECT */9 COLUMNS

## Calculate Mandelbrot Channel


In [14]:
from humbldata.toolbox.technical.humbl_channel.model import calc_humbl_channel


mandelbrot = calc_humbl_channel(
    data,
    window="1m",
    rv_adjustment=True,
    rv_method="yz",
    rv_grouped_mean=False,
    rs_method="RS",
    live_price=True
).collect()


In [13]:
mandelbrot

date,symbol,bottom_price,recent_price,top_price
date,str,f64,f64,f64
2023-12-29,"""AAPL""",216.1965,216.67,225.1068
2023-12-29,"""AMD""",147.4549,158.4,161.7655
2023-12-29,"""AMZN""",180.0482,184.06,189.3734
2023-12-29,"""GOOGL""",171.1965,177.24,179.6634
2023-12-29,"""PCT""",3.5021,5.98,7.6052


## Find Prices Near the Top/Bottom of the Channel

In [ ]:
mandelbrot_historical.filter(
    (pl.col("close") >= pl.col("top_price") * 0.975)
    & (pl.col("close") <= pl.col("bottom_price") * 1.025)
)
